In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def process_wastewater_data(file_path):
    
    # Load wastewater data
    wastewater_data = pd.read_csv(file_path)
    
    # Mapping dictionaries
    country_mapping_ww = {
        'AT': 'Austria',
        'BE': 'Belgium',
        'BG': 'Bulgaria',
        'CH': 'Switzerland',
        'CY': 'Cyprus',
        'CZ': 'Czech Republic',
        'DE': 'Germany',
        'DK': 'Denmark',
        'EE': 'Estonia',
        'ES': 'Spain',
        'FI': 'Finland',
        'FR': 'France',
        'GB': 'United Kingdom',
        'GR': 'Greece',
        'HR': 'Croatia',
        'HU': 'Hungary',
        'IE': 'Ireland',
        'IS': 'Iceland',
        'IT': 'Italy',
        'LT': 'Lithuania',
        'LU': 'Luxembourg',
        'LV': 'Latvia',
        'MT': 'Malta',
        'NL': 'Netherlands',
        'NO': 'Norway',
        'PL': 'Poland',
        'PT': 'Portugal',
        'RO': 'Romania',
        'RS': 'Serbia',
        'SE': 'Sweden',
        'SI': 'Slovenia',
        'SK': 'Slovakia',
        'TR': 'Turkey',
        'UK': 'United Kingdom'
    }

    wastewater_drugs_mapping = {
        'cocaine': 'Cocaine',
        'cannabis': 'Cannabis'    
    }

    # Map the country abbreviations to full names in wastewater_data
    wastewater_data['Country'] = wastewater_data['Country'].replace(country_mapping_ww)
    wastewater_data['Metabolite'] = wastewater_data['Metabolite'].replace(wastewater_drugs_mapping)
    wastewater_data = wastewater_data.drop(columns=['source'])
    
    # Ensure the column names are consistent
    wastewater_data.rename(columns={'Country': 'country', 'Year': 'year', 'Metabolite': 'drug'}, inplace=True)
    
    # Filter the data
    wastewater_data = wastewater_data[wastewater_data['year'] >= 2019]
    wastewater_data = wastewater_data[wastewater_data['drug'].isin(['Cocaine', 'Cannabis', 'MDMA'])]
    
    return wastewater_data


In [ ]:
def process_forecasting_data():
    country_mapping = {
        'GERMANY': 'Germany',
        'ITALY': 'Italy',
        'POLAND': 'Poland',
        'UK': 'United Kingdom',
        'BELGIUM': 'Belgium',
        'CZECHIA': 'Czech Republic',
        'DANISH': 'Denmark',
        'FRANCE': 'France',
        'IRELAND': 'Ireland',
        'LUXEMBOURG': 'Luxembourg',
        'NETHERLANDS': 'Netherlands',
        'PORTUGAL': 'Portugal',
        'SPAIN': 'Spain',
        'SWEDEN': 'Sweden',
        'FRENCH': 'France',
        'DENMARK': 'Denmark'
    }

    drugs_mapping = {
        'COCAINE': 'Cocaine',
        'marijuana': 'Cannabis'
    }

    def process_data(file, years):
        data = pd.read_csv(file)
        data['drug'] = data['drug'].replace(drugs_mapping)
        data['country'] = data['country'].replace(country_mapping)
        return data[(data['drug'].isin(['Cocaine', 'Cannabis', 'MDMA'])) & (data['year'].isin(years))][['year', 'country', 'drug', 'total']]

    # Process and merge data
    reddit_data = pd.concat([process_data('reddit_formatted.csv', [2020, 2021, 2022, 2023]), process_data('finish_for_david.csv', [2019])])

    return reddit_data


In [ ]:
def process_combined_data(reddit_data, wastewater_data):
    filtered_reddit_data = reddit_data[reddit_data['year'] == 2023]
    filtered_reddit_data['litre/day per 1 000 inhabitants'] = np.nan

    drug_list = wastewater_data['drug'].unique().tolist()

    filtered_reddit_data = filtered_reddit_data[filtered_reddit_data['drug'].isin(drug_list)]

    combined_data = pd.concat([filtered_reddit_data, combined_data])
    combined_data = combined_data.drop_duplicates()

    counts = combined_data.groupby(['country', 'drug']).size()
    max_count = counts.max()

    filtered_counts = counts[counts == max_count]

    combined_data = combined_data.set_index(['country', 'drug']).loc[filtered_counts.index].reset_index()
    combined_data = combined_data.sort_values(by=['country', 'drug', 'year']).reset_index(drop=True)

    combined_data.to_csv('combined_drug_data.csv')


In [ ]:
def clean_drug_trends_data(file_path):
    
    # Load the data
    data = pd.read_csv(file_path)

    # Filter by specified countries
    countries = ['Belgium', 'Czech Republic', 'Germany', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Spain', 'Sweden']
    data = data[data['Country'].isin(countries)]

    # Filter by specified substances
    substances = ['Cannabis', 'Cocaine', 'Ecstasy']
    data = data[data['Substance'].isin(substances)]

    # Rename 'Ecstasy' to 'MDMA'
    data['Substance'] = data['Substance'].replace('Ecstasy', 'MDMA')

    # Rename 'Substance' column to 'drug'
    data.rename(columns={'Substance': 'drug'}, inplace=True)

    # Replace empty strings with NaN
    data.replace('', np.nan, inplace=True)
    data.replace(' ', np.nan, inplace=True)  # In case there are spaces

    # Keep rows with 'Last year' and 'Adults (15-64)'
    data = data[(data['Recall period'] == 'Last year') & (data['Age'] == 'Adults (15-64)')]

    # Remove specified columns
    columns_to_drop = ['Country code', 'Geographical scope', 'Recall period', 'Age']
    data.drop(columns=columns_to_drop, inplace=True)

    return data


In [ ]:
def clean_forecasting_data(file_path):
    # Load the data
    data = pd.read_csv(file_path)
    data = data.drop(columns=['source'])

    # Filter rows based on 'Country'
    countries_to_keep = ['Belgium', 'Czech Republic', 'Germany', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Spain', 'Sweden']
    data = data[data['Country'].isin(countries_to_keep)]

    # Filter rows based on 'Substance'
    substances_to_keep = ['cannabis', 'cocaine', 'MDMA']
    data = data[data['Metabolite'].isin(substances_to_keep)]

    # Rename 'Ecstasy' to 'MDMA' in 'Substance'
    data['Metabolite'] = data['Metabolite'].replace('cannabis', 'Cannabis')
    data['Metabolite'] = data['Metabolite'].replace('cocaine', 'Cocaine')

    # Rename the column 'Substance' to 'drug'
    data = data.rename(columns={'Metabolite': 'drug'})

    data.to_csv('un_data_cleaned.csv', index=False)


In [ ]:
def pivot_un_data(file_path, output_file_path):
    # Load the CSV file into a DataFrame
    data = pd.read_csv(file_path)

    # Pivot the DataFrame
    pivoted_data = data.pivot_table(index=['Country', 'drug'], columns='Year', values='Percentage of the population', aggfunc='first').reset_index()

    # Save the pivoted DataFrame back to a CSV file
    pivoted_data.to_csv(output_file_path, index=False)
file_path = 'un_data_cleaned.csv'  # Update this path if needed
data = pd.read_csv(file_path)

# Pivot the DataFrame
pivoted_data = data.pivot_table(index=['Country', 'drug'], columns='Year', values='Percentage of the population', aggfunc='first').reset_index()

# Save the pivoted DataFrame back to a CSV file if needed
output_file_path = 'pivoted_un_data.csv'
pivoted_data.to_csv(output_file_path, index=False)

In [ ]:
import pandas as pd
def merge_and_clean_data(drug_trends_file, pivoted_un_data_file, output_file):
    # Load the CSV files into DataFrames
    drug_trends_df = pd.read_csv(drug_trends_file)
    pivoted_un_data_df = pd.read_csv(pivoted_un_data_file)

    # Merge the DataFrames on 'Country' and 'drug' columns using a left join
    merged_df = pd.merge(drug_trends_df, pivoted_un_data_df, on=['Country', 'drug'], how='left', suffixes=('', '_enrich'))

    # For each year with an "_enrich" column, update the main column if it's NaN and the "_enrich" column has a value.
    for year in range(2004, 2022):  # Loop through the years that have enrichment data
        year_col = str(year)
        enrich_col = f"{year_col}_enrich"
        # Where the main year column is NaN and the enrich column has a value, update the main column
        merged_df[year_col] = merged_df.apply(lambda row: row[enrich_col] if pd.isna(row[year_col]) and not pd.isna(row[enrich_col]) else row[year_col], axis=1)

    # Drop the "_enrich" columns after updating
    columns_to_drop = [f"{year}_enrich" for year in range(2004, 2022)]
    merged_df.drop(columns=columns_to_drop, inplace=True)

    # Save the merged and cleaned DataFrame to a CSV file
    merged_df.to_csv(output_file, index=False)


In [ ]:
from sklearn.linear_model import LinearRegression

def generate_synthetic_values(group, years_to_predict=[2019, 2020, 2021, 2022]):
    numeric_values = group.select_dtypes(include=[np.number]).values.flatten()
    years = np.arange(1990, 2023)  # Corresponding years
    
    available_data_mask = ~np.isnan(numeric_values)
    available_years = years[available_data_mask]
    available_values = numeric_values[available_data_mask]
    
    results = []
    if len(available_years) >= 2:
        X = available_years.reshape(-1, 1)
        y = available_values
        model = LinearRegression()
        model.fit(X, y)
        
        predictions = model.predict(np.array(years_to_predict).reshape(-1, 1))
        for year, value in zip(years_to_predict, predictions):
            results.append((year, value))
    
    return results

def main_synthetic():
    # Load your data
    file_path = 'merged_un_emcdd.csv'  # Change this to your actual file path
    data = pd.read_csv(file_path)
    
    synthetic_results = []
    for (country, drug), group in data.groupby(['Country', 'drug']):
        predictions = generate_synthetic_values(group)
        for year, value in predictions:
            synthetic_results.append({'Country': country, 'Drug': drug, 'Year': year, 'Value': value})

    synthetic_data = pd.DataFrame(synthetic_results)
    synthetic_data.to_csv('merged_synthetic_data.csv', index=False)  # Save to file
    return synthetic_data

In [ ]:
def update_combined_drug_data(merged_synthetic_data_path, combined_drug_data_path, updated_combined_file_path):
    merged_synthetic_data = pd.read_csv(merged_synthetic_data_path)
    combined_drug_data = pd.read_csv(combined_drug_data_path)

    # Rename the 'Value' column to 'percentage of users last year'
    merged_synthetic_data.rename(columns={'Value': 'percentage of users last year'}, inplace=True)

    # Correcting the merge operation to use lowercase column names to match the combined drug data
    # Assume the combined drug data has columns named 'country', 'drug', 'year'
    updated_combined_data = pd.merge(combined_drug_data, 
                                      merged_synthetic_data[['Country', 'Drug', 'Year', 'percentage of users last year']],
                                      left_on=['country', 'drug', 'year'], 
                                      right_on=['Country', 'Drug', 'Year'], 
                                      how='left')

    # Drop the duplicate columns from the merge operation, if necessary
    updated_combined_data.drop(columns=['Country', 'Drug', 'Year'], inplace=True, errors='ignore')

    # Save the updated dataframe to a new CSV file
    updated_combined_data.to_csv(updated_combined_file_path, index=False)

    print(f"Updated combined drug data saved to: {updated_combined_file_path}")
